In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf
import pandas as pd
from sklearn import model_selection

2023-01-31 04:41:19.799082: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-31 04:41:19.800347: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-31 04:41:19.801165: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-31 04:41:19.804049: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [2]:
train = pd.read_csv("/kaggle/input/c-code-snippets-and-their-labels/train.csv")
test = pd.read_csv("/kaggle/input/c-code-snippets-and-their-labels/test.csv")

In [3]:
train

,code,labels
0,"#include <stdio.h>\n\nint main(int argc, const...",0
1,#include <stdio.h>\nint main(void)\n{\n\tint a...,1
2,#include<stdio.h>\n\nint main(void)\n{\n\n\tch...,0
3,/*\nexit\n*/\n#include <stdio.h>\n#include <st...,0
4,"#include<stdio.h>\nint main(void){\n\tint S,h,...",0
...,...,...
326314,#include<stdio.h>\n\nint main(void)\n{\n\tint ...,0
326315,"#include<stdio.h>\nmain(){\n int a,b,c,d;\n ...",1
326316,#include <stdio.h>\n\nint main(void){\n\tint X...,0
326317,#include<stdio.h>\n#include<stdlib.h>\n#includ...,0


In [4]:
test

,code,labels
0,#include<stdio.h>\n#include<string.h>\n\n#defi...,1
1,"s=""%d %d %d %d %d\n"";main(a,b,c,d,e){qsort(&a,...",1
2,"#include<stdio.h>\nint main()\n{\n\tint d,n,x,...",0
3,#include <stdio.h>\n\nint main(void){\n \n ...,1
4,#include<stdio.h>\n#include<string.h>\n\nint m...,0
...,...,...
36252,"double M[12][4096],r,t;D[12],n;\ndouble dfs(p,...",0
36253,"c,v,s;\nmain(){\n\tfor(;c=getchar(),c+1;){\n\t...",0
36254,/*\n問題内容：\n某国に住んでいる友達がお金に困って、あるヤミ金融業者から 10 万円の...,0
36255,"#include<stdio.h>\nvoid swap(int *a,int *b){\n...",1


In [ ]:
classifier = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
classifier.summary()

In [ ]:
InputExample(guid=None,
             text_a = "Hello, world",
             text_b = None,
             label = 1)

In [ ]:
def convert_data_to_examples(train, test, code_snippet, label_of_code_snippet): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[code_snippet], 
                                                          text_b = None,
                                                          label = x[label_of_code_snippet]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[code_snippet], 
                                                          text_b = None,
                                                          label = x[label_of_code_snippet]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'code', 
                                                                           'labels')

In [ ]:
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=256):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )



In [ ]:
code_snippet = 'code'
label_of_code_snippet = 'labels'

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, code_snippet, label_of_code_snippet)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

In [ ]:
# classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
#               metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

In [ ]:
# classifier.fit(train_data, epochs=7, 
#           validation_data=validation_data,
#           verbose=1,batch_size=256,
#           steps_per_epoch=1400)

In [ ]:
#Hyperparameter tuning using grid search

In [ ]:
param_grid = {
    "per_gpu_batch_size": [128, 256,512],
    "learning_rate": [2e-5, 3e-5, 5e-5],
    "num_epochs": [2, 4, 7,10]
}

In [ ]:
model = model_selection.RandomizedDearchCV(
estimator=classifier,
    param_grid=param_grid,
    scoring="accuracy",
    verbose=10,
    n_jobs=2,
    cv=5
)

In [ ]:
model.fit(train_data)